In [1]:
import panel as pn
import mysql.connector
import re
from quiz_questions import get_quiz_data
from pathlib import Path


pn.extension()

# ========== Database Connection ==========
def connect_quiz_db():
    try:
        return mysql.connector.connect(
            host="localhost",
            user="root",
            password="",
            database="quiz_game"
        )
    except mysql.connector.Error as err:
        print(f"Database connection error: {err}")
        return None

# ========== Global State ==========
class QuizState:
    def __init__(self):
        self.username = ""
        self.selected_category = None
        self.questions = []
        self.question_index = 0
        self.score = 0
        
    def reset(self):
        self.__init__()

quiz_state = QuizState()

# ========== Widget Definitions ==========
# Admin Widgets
admin_user = pn.widgets.TextInput(name="Admin Username")
admin_pass = pn.widgets.PasswordInput(name="Admin Password")
admin_confirm_pass = pn.widgets.PasswordInput(placeholder="Confirm Password", width=250)
admin_login_button = pn.widgets.Button(name="Login", button_type="primary")
admin_status = pn.pane.Markdown("", width=400)

# Question Management Widgets
question_category = pn.widgets.TextInput(name="Category")
question_text = pn.widgets.TextInput(name="Question")
option1 = pn.widgets.TextInput(name="Option 1")
option2 = pn.widgets.TextInput(name="Option 2")
option3 = pn.widgets.TextInput(name="Option 3")
option4 = pn.widgets.TextInput(name="Option 4")
correct_answer = pn.widgets.TextInput(name="Correct Answer")
add_question_button = pn.widgets.Button(name="Add Question", button_type="success")
question_status = pn.pane.Markdown("", width=400)

# Player Widgets
name_input = pn.widgets.TextInput(name="Enter Your Name")
category_dropdown = pn.widgets.Select(name="Select Category", options=["Select Category"] + list(get_quiz_data().keys()))
start_button = pn.widgets.Button(name="Start Quiz", button_type="success", width=200)
question_pane = pn.pane.Markdown("", align="center")
status_pane = pn.pane.Markdown("", align="center")
option_buttons = [pn.widgets.Button(name=f"Option {i+1}", button_type="primary", width=400, visible=False) for i in range(4)]
return_home_btn = pn.widgets.Button(name="Return to Main Page", button_type="success", width=200)

# Navigation Buttons
player_login_btn = pn.widgets.Button(name="🎮 Player Login", button_type="primary", width=200)
admin_login_btn = pn.widgets.Button(name="🛠️ Admin Login", button_type="success", width=200)


# ========== Quiz Functions ==========
option_buttons = []  # ✅ Store buttons globally

def start_quiz(event=None):
    """ Initialize quiz with selected category and username """
    quiz_state.username = name_input.value.strip()
    quiz_state.selected_category = category_dropdown.value

    if not quiz_state.username or not re.match(r"^[A-Za-z\s'-]+$", quiz_state.username):
        status_pane.object = "**Invalid name! Only letters, spaces, hyphens, and apostrophes allowed.**"
        return
    if quiz_state.selected_category == "Select Category":
        status_pane.object = "**Please select a category!**"
        return
    
    quiz_state.questions = get_quiz_data().get(quiz_state.selected_category, [])  # ✅ Fetching questions from `quiz_questions`
    quiz_state.question_index = 0
    quiz_state.score = 0

    current_view.objects = [quiz_app]
    show_question()

import functools

def show_question():
    """ Display the current question and options """
    if quiz_state.question_index < len(quiz_state.questions):
        q, options, correct_answer = quiz_state.questions[quiz_state.question_index]

        # ✅ Styled question text
        question_html = f"""
        <div style='text-align:center; font-size:24px; font-weight:bold; 
                    font-family:"Garamond", "Poppins", sans-serif; color:#2C3539;
                    background: linear-gradient(to left, #800020, #333333);
                    -webkit-background-clip: text; 
                    -webkit-text-fill-color: transparent;
                    margin-bottom:20px;'>
            {q}
        </div>
        """
        question_pane.object = question_html

        # ✅ Clear old buttons & create new ones
        global option_buttons
        option_buttons.clear()

        for i, option in enumerate(options):
            btn = pn.widgets.Button(
               name=option, 
                button_type="primary", 
                width=200,  # 🔥 Wider for better look
                height=45,  # 🔥 Taller for more emphasis
                styles={
                    "font-size": "60px",  # 🔥 SUPER BIG TEXT
                    "font-weight": "1000",  # 🔥 MAXIMUM BOLDNESS
                    "font-family": "'Poppins', 'Comic Sans MS'",  # 🔥 Elegant & Fun Font
                    "text-align": "center",  # 🔥 Ensures text is centered
                    "letter-spacing": "2px"  # 🔥 Slight spacing for readability
                }
            )
            btn.on_click(lambda event, idx=i: check_answer(idx, correct_answer))  # ✅ Correctly passing index & answer
            option_buttons.append(btn)

        # ✅ Styled option layout
        options_layout = pn.Column(
            *option_buttons, 
            align="center", 
            styles={"display": "flex", "flex-direction": "column", "gap": "15px"}
        )

        # ✅ Box for all components (background color + styling)
        question_box = pn.Column(
            question_pane,  
            options_layout,  
            align="center",
            styles={
                "background-color": "#F5F5DC",  # 🔥 Soft pinkish-beige
                "padding": "40px",
                "border-radius": "20px",
                "box-shadow": "5px 5px 15px gray",
                "width": "480px",  
            }
        )

        # ✅ Full-page centering with right shift
        current_view.objects = [
            pn.Row(  
                pn.Spacer(width=350),  # 🔥 Moves the box to the right (Adjust value as needed)
                question_box,  
                pn.layout.VSpacer(),
                align="center",
                styles={"height": "100vh"}  
            )
        ]

    else:
        show_score()  # ✅ Show final score after last question



def check_answer(selected_index, correct_answer):
    """ Check the selected answer and update score """
    q, options, _ = quiz_state.questions[quiz_state.question_index]

    # ✅ Compare selected answer text instead of index
    if options[selected_index] == correct_answer:
        quiz_state.score += 1  # ✅ Increase score if correct

    quiz_state.question_index += 1  # ✅ Move to next question
    print(f"Current Score: {quiz_state.score}")  # ✅ Debugging
    show_question()  # ✅ Load next question


def save_score():
    """ Save player's score to the database """
    print(f"Saving Score: {quiz_state.score}")  # ✅ Debugging
    conn = connect_quiz_db()
    if conn:
        try:
            cursor = conn.cursor()
            cursor.execute("""
                INSERT INTO leaderboard (player_name, category, score) 
                VALUES (%s, %s, %s) 
                ON DUPLICATE KEY UPDATE score = GREATEST(score, %s)
                """, 
                (quiz_state.username, quiz_state.selected_category, quiz_state.score, quiz_state.score))
            conn.commit()
        except Exception as err:
            print(f"Database Error: {err}")
        finally:
            conn.close()
def show_score():
    """ Display final score and performance feedback in a well-styled layout """
    print(f"Final Score: {quiz_state.score}")  # ✅ Debugging
    
    # ✅ Style the score display
    question_pane.object = f"""
        <div style="text-align:center; font-size:45px; font-weight:bold; font-family:'Poppins', Garamond; color:#4B0082;">
            🎯 Your Score: {quiz_state.score}
        </div>
    """

    # ✅ Determine performance message with colors
    if quiz_state.score >= 4:
        performance = "<span style='color:#191970;'>🌟 Excellent! 🎉</span>"
    elif quiz_state.score == 3:
        performance = "<span style='color:#32CD32;'>👍 Good Job!</span>"
    elif quiz_state.score == 2:
        performance = "<span style='color:#FFA500;'>😊 Not bad, keep trying!</span>"
    else:
        performance = "<span style='color:#DC143C;'>😞 Better luck next time!</span>"

    status_pane.object = f"""
        <div style="text-align:center; font-size:30px; font-weight:bold; font-family:'Poppins', Garamond;">
            {performance}
        </div>
    """

    # ✅ Hide option buttons
    for btn in option_buttons:
        btn.visible = False

    # ✅ Save the score in the database
    save_score()

    # ✅ Create buttons with bigger font and better styling
    return_home_btn = pn.widgets.Button(
        name="🏠 Return to Main Menu", button_type="warning", width=300, height=60,
        styles={"font-size": "28px", "font-weight": "bold", "font-family": "'Poppins', Comic Sans MS"}
    )
    return_home_btn.on_click(return_home)

    view_top_10_btn = pn.widgets.Button(
        name="🏆 View Top 10 Players", button_type="success", width=300, height=60,
        styles={"font-size": "28px", "font-weight": "bold", "font-family": "'Poppins', Comic Sans MS"}
    )
    view_top_10_btn.on_click(show_top_players)

    # ✅ Updated layout with vertical spacing and centered position
    score_layout = pn.Column(
        pn.layout.VSpacer(),  # Pushes content downward
        question_pane, 
        status_pane,
        pn.Spacer(height=30),  # Adds spacing between text and buttons
        view_top_10_btn, 
        return_home_btn, 
        pn.layout.VSpacer(),  # Pushes content upward
        align="center",
        styles={
            "background": "#D3D3D3",  # Light beige background for the box
            "border": "3px solid #6A5ACD",  # Lavender border
            "border-radius": "15px",
            "padding": "30px",
            "box-shadow": "5px 5px 15px gray",
            "font-family": "Poppins, Garamond",
            "margin": "auto",  # Centers horizontally
            "display": "flex",
            "flex-direction": "column",
            "align-items": "center",
            "width": "500px"
        }
    )

    # ✅ Apply full-page background styling
    background = pn.pane.HTML(f"""
        <div style="position: fixed; width: 100%; height: 100%; background: url('Documents/Python lab programs/images/browsebg.jpg') no-repeat center center/cover;">
        </div>
    """)

    # ✅ Refresh UI with new layout
    current_view.objects = [background, pn.Row(pn.Spacer(width=350), score_layout, pn.Spacer(width=350))]

def return_home(event=None):
    """Resets the UI and returns to the main menu."""
    current_view.objects.clear()  # ✅ Clear quiz view
    current_view.objects = [main_menu]  # ✅ Show the main menu

def show_top_players(event=None):
    """ Display the Top 10 Players in a styled box with bold colors for the top 3 """
    conn = connect_quiz_db()
    if not conn:
        status_pane.object = "**Database connection failed. Try again later.**"
        return

    try:
        cursor = conn.cursor()
        cursor.execute("SELECT player_name, score FROM leaderboard ORDER BY score DESC, player_name ASC LIMIT 10")
        top_players = cursor.fetchall()
    except Exception as err:
        print(f"Database Error: {err}")
        status_pane.object = "**Error retrieving leaderboard.**"
        return
    finally:
        conn.close()

    if not top_players:
        status_pane.object = "**No players in the leaderboard yet!**"
        return

    # ✅ Styling for leaderboard box (white) & outer container (beige)
    leaderboard_html = """
    <div style='text-align:center; padding:15px; border:3px solid #333; border-radius:10px; background:white ; width:100%; max-width:450px; margin:auto;'>
        <h2 style='font-size:38px; font-family:Garamond, sans-serif; color:#222; margin-bottom:10px;'>🏆 Top 10 Players 🏆</h2>
    """

    for i, (name, score) in enumerate(top_players):
        color = "#FFD700" if i == 0 else "#C0C0C0" if i == 1 else "#CD7F32" if i == 2 else "#353839"
        font_weight = "bold" if i < 3 else "normal"
        leaderboard_html += f"""
        <p style='font-size:{30 if i < 3 else 25}px; font-family:Garamond, sans-serif; color:{color}; font-weight:{font_weight}; margin:5px 0;'>
            {i+1}. {name} - {score} pts
        </p>
        """

    leaderboard_html += "</div>"

    # ✅ Display leaderboard inside a beige background panel
    leaderboard_pane = pn.pane.HTML(leaderboard_html, width=450)

    # ✅ Create 'Return' button (outside the box)
    return_home_btn = pn.widgets.Button(
        name="🏠 Return to Main Menu", button_type="warning", width=250,
        styles={"font-size": "22px", "font-weight": "bold"}
    )
    return_home_btn.on_click(return_home)

    # ✅ Wrap everything in a beige background & move right with Spacer
    beige_background = pn.Row(
        pn.Spacer(width=400),  # ✅ Moves leaderboard box to the right
        pn.Column(
            pn.Spacer(height=50),  # ✅ Pushes the box downward
            pn.pane.HTML("<div style='background:#A99E80; padding:25px; border-radius:15px; text-align:center;'>"),  # Beige outer area
            leaderboard_pane,
            return_home_btn,
            pn.pane.HTML("</div>"),
            align="center"
        )
    )

    # ✅ Update view with the styled leaderboard
    current_view.objects = [beige_background]


# ========== Admin Functions ==========
def admin_login(event=None):
    ensure_admin_table()

    username = admin_user.value.strip()
    password = admin_pass.value.strip()
    confirm_password = admin_confirm_pass.value.strip()

    if password != confirm_password:
        admin_status.object = "❌ Passwords do not match!"
        return

    conn = connect_quiz_db()
    if conn:
        try:
            cursor = conn.cursor()
            cursor.execute("SELECT username, password FROM admin_profile WHERE id=1")
            result = cursor.fetchone()
            if result:
                db_username, db_password = result
                if username == db_username and password == db_password:
                    admin_status.object = "✅ Login Successful! Redirecting..."
                    show_admin_dashboard()
                else:
                    admin_status.object = "❌ Invalid Credentials. Try Again!"
            else:
                admin_status.object = "❌ No admin found in the database!"
        except mysql.connector.Error as err:
            admin_status.object = f"❌ Database Error: {err}"
        finally:
            conn.close()
    else:
        admin_status.object = "❌ Database Connection Failed!"

def show_admin_dashboard():
    # Header with welcome message and logout
    header = pn.Row(
        pn.pane.Markdown(
            "## 🧑‍💼 Welcome! Enjoy managing Quiziverse..",
            styles={
                "font-size": "26px",
                "font-family": "Garamond",
                "font-weight": "bold",
                "padding": "10px"
            },
            width=800
        ),
        pn.layout.HSpacer(),
        pn.widgets.Button(
            name="🚪 Logout",
            button_type="danger",
            width=120,
            styles={"font-size": "16px", "font-family": "Comic Sans MS"},
            on_click=lambda event: show_welcome()
        ),
        styles={"margin-bottom": "20px", "background-color": "#E8E8E8", "padding": "10px", "border-radius": "10px", "box-shadow": "2px 2px 8px gray"}
    )

    # Admin control buttons
    admin_controls = pn.Column(
        pn.Spacer(height=10),
        pn.widgets.Button(
            name="✏️ Update Profile", 
            button_type="primary", 
            width=220, 
            styles={"font-size": "18px", "font-family": "Comic Sans MS"},
            on_click=lambda event: show_update_profile()
        ),
        pn.Spacer(height=10),
        pn.widgets.Button(
            name="📋 Manage Questions", 
            button_type="success", 
            width=220, 
            styles={"font-size": "18px", "font-family": "Comic Sans MS"},
            on_click=lambda event: show_manage_questions()
        ),
        pn.Spacer(height=10),
        pn.widgets.Button(
            name="👥 Manage Players", 
            button_type="warning", 
            width=220, 
            styles={"font-size": "18px", "font-family": "Comic Sans MS"},
            on_click=lambda event: manage_players()
        ),
        align="start",
        width=260,
        styles={"background-color": "#F5F5F5", "padding": "20px", "border-radius": "15px", "box-shadow": "5px 5px 15px gray"}
    )

    # Insights section
    insights_section = pn.Column(
        pn.pane.Markdown("### 📊 Quiz Insights", styles={"font-size": "24px", "font-family": "Garamond", "font-weight": "bold"}),
        pn.Spacer(height=10),
        pn.Row(
            pn.pane.Matplotlib(get_category_pie_chart(), tight=True, width=400, height=350),
            pn.pane.Matplotlib(get_player_bar_chart(), tight=True, width=400, height=350),
            align="center"
        ),
        align="center",
        styles={"background-color": "#FFFFFF", "padding": "20px", "border-radius": "15px", "box-shadow": "5px 5px 15px gray"}
    )

    # Full dashboard layout
    admin_dashboard = pn.Column(
        header,
        pn.Row(
            admin_controls,
            pn.Spacer(width=30),
            insights_section
        ),
        sizing_mode='stretch_width'
    )

    current_view.objects = [admin_dashboard]



def get_category_pie_chart():
    import matplotlib.pyplot as plt
    conn = connect_quiz_db()
    category_counts = {}
    
    if conn:
        try:
            cursor = conn.cursor()
            cursor.execute("SELECT category, COUNT(*) FROM leaderboard GROUP BY category")
            category_counts = dict(cursor.fetchall())
        except Exception as e:
            print(f"Error fetching category data: {e}")
        finally:
            conn.close()
    
    fig, ax = plt.subplots()
    ax.pie(category_counts.values(), labels=category_counts.keys(), autopct='%1.1f%%', startangle=140, colors=['#ff9999','#66b3ff','#99ff99','#ffcc99'])
    ax.set_title("Players Per Category")
    return fig

def get_player_bar_chart():
    import matplotlib.pyplot as plt
    conn = connect_quiz_db()
    player_scores = {}
    
    if conn:
        try:
            cursor = conn.cursor()
            cursor.execute("SELECT player_name, score FROM leaderboard ORDER BY score DESC LIMIT 10")
            results = cursor.fetchall()
            player_scores = {name: score for name, score in results}
        except Exception as e:
            print(f"Error fetching player data: {e}")
        finally:
            conn.close()
    
    fig, ax = plt.subplots()
    ax.bar(player_scores.keys(), player_scores.values(), color='#66b3ff')
    ax.set_xlabel("Players")
    ax.set_ylabel("Score")
    ax.set_title("Top 10 Players by Score")
    plt.xticks(rotation=45, ha='right')
    return fig


def ensure_admin_table():
    """Creates the admin_profile table if it doesn't exist."""
    conn = connect_quiz_db()
    if conn:
        try:
            cursor = conn.cursor()
            cursor.execute("""
                CREATE TABLE IF NOT EXISTS admin_profile (
                    id INT PRIMARY KEY AUTO_INCREMENT,
                    username VARCHAR(50) NOT NULL UNIQUE,
                    password VARCHAR(100) NOT NULL
                )
            """)
            # Ensure default admin credentials exist
            cursor.execute("SELECT COUNT(*) FROM admin_profile")
            count = cursor.fetchone()[0]
            if count == 0:
                cursor.execute("INSERT INTO admin_profile (username, password) VALUES ('admin', 'admin123')")
                conn.commit()
        except mysql.connector.Error as err:
            print(f"Database Error: {err}")
        finally:
            conn.close()


import re  # Import regex for validation

def show_update_profile():
    """Displays the update profile page for the admin."""
    
    # Input fields
    new_username = pn.widgets.TextInput(name="👤 New Username", placeholder="Enter new username", width=300)
    new_password = pn.widgets.PasswordInput(name="🔑 New Password", placeholder="Enter new password", width=300)
    
    # Status message pane
    update_status = pn.pane.Markdown("", styles={"font-size": "16px", "color": "green"})

    # Function to validate and update profile
    def validate_and_update(event):
        """Validates input and updates the profile."""
        username = new_username.value.strip()
        password = new_password.value.strip()

        # Attempt to update profile and show message
        message = update_admin_profile(username, password)
        update_status.object = f"**{message}**"  # Display success or error message in bold
    
    # Update button
    update_btn = pn.widgets.Button(name="💾 Save Changes", button_type="success", width=200)
    update_btn.on_click(validate_and_update)  # Call function on click

    # Go back button
    back_btn = pn.widgets.Button(name="🔙 Back to Dashboard", button_type="warning", width=200)
    back_btn.on_click(show_admin_dashboard)  # Ensure this works

    def go_back(event):
        show_admin_dashboard()  # Ensure the dashboard is displayed
    back_btn = pn.widgets.Button(name="🔙 Back to Dashboard", button_type="warning", width=200)
    back_btn.on_click(go_back)  # Explicitly set event handler

    # Layout
    update_profile_page = pn.Column(
        pn.pane.Markdown("## ✏️ Update Admin Profile", styles={"font-size": "24px", "font-family": "Garamond", "font-weight": "bold", "text-align": "center"}),
        pn.Spacer(height=10),
        new_username,
        pn.Spacer(height=10),
        new_password,
        pn.Spacer(height=20),
        update_btn,
        back_btn,
        pn.Spacer(height=20),
        update_status,  # Display update message here
        align="center",
        styles={"background-color": "#AEC6CF", "padding": "20px", "border-radius": "15px", "box-shadow": "5px 5px 15px gray"}
    )

    current_view.objects = [update_profile_page]  # Update view with profile page

def update_admin_profile(new_username, new_password):
    """Updates the admin's username and password in the database with validation."""
    if not new_username or not new_password:
        return "❌ Username and Password cannot be empty!"
    
    if not new_username.isalpha():
        return "❌ Username should contain only alphabets!"
    
    conn = connect_quiz_db()
    if conn:
        try:
            cursor = conn.cursor()
            cursor.execute("SELECT username, password FROM admin_profile WHERE id=1")
            existing_admin = cursor.fetchone()

            if existing_admin:
                old_username, old_password = existing_admin
                
                # Check if both username and password are unchanged
                if new_username == old_username and new_password == old_password:
                    return "⚠️ You entered the same username and password!"
                
                # Update only if changed
                cursor.execute("UPDATE admin_profile SET username=%s, password=%s WHERE id=1", 
                               (new_username, new_password))
                conn.commit()
                return "✅ Profile updated successfully!"
            
            return "❌ Admin profile not found!"
        
        except mysql.connector.Error as err:
            return f"❌ Database Error: {err}"
        
        finally:
            conn.close()



def save_admin_profile(new_username, new_password):
    conn = connect_quiz_db()
    if conn:
        try:
            cursor = conn.cursor()
            cursor.execute("UPDATE admin SET username=%s, password=%s WHERE id=1", (new_username, new_password))
            conn.commit()
            return "✅ Profile updated successfully!"
        except mysql.connector.Error as err:
            return f"❌ Database Error: {err}"
        finally:
            conn.close()
    return "❌ Failed to update profile. Please try again."

def show_manage_questions():
    """Displays the Manage Questions page with options to View, Add, Edit, or Delete questions."""
    
    manage_questions_page = pn.Column(
        pn.pane.Markdown(
            "## ❓ Manage Questions 📝",
            styles={"font-size": "30px", "font-family": "Garamond", "font-weight": "bold", "text-align": "center"}
        ),
        pn.Spacer(height=20),
        pn.Column(
            pn.widgets.Button(
                name="📜 View All Questions", 
                button_type="primary", 
                width=300, 
                on_click=lambda event: show_all_questions()
            ),
            pn.Spacer(height=10),
            pn.widgets.Button(
                name="➕ Add a Question",
                button_type="success",
                width=300,
                on_click=lambda event: show_add_question()  # Calls the function
            ),
            pn.Spacer(height=10),
            pn.widgets.Button(
                name="✏️ Edit a Question", 
                button_type="warning", 
                width=300, 
                on_click=lambda event: show_edit_question()
            ),
            pn.Spacer(height=10),
            pn.widgets.Button(
                name="❌ Delete a Question", 
                button_type="danger", 
                width=300, 
                on_click=lambda event: show_delete_question()
            ),
            pn.Spacer(height=20),
             pn.widgets.Button(
                name="🔙 Back to Dashboard", 
                button_type="light", 
                width=300, 
                on_click=lambda event: show_admin_dashboard()  # Ensures correct navigation
            ),
            align="center",
            styles={"background-color": "#F5F5F5", "padding": "20px", "border-radius": "15px", "box-shadow": "5px 5px 15px gray"}
        ),
        align="center"
    )

    current_view.objects = [manage_questions_page]


import panel as pn

def show_all_questions():
    """Displays all quiz questions from the database."""
    
    # Fetch questions from the database
    db_questions = get_db_questions()

    # Debugging: Print the number of questions
    print(f"🔍 Total Questions Retrieved: {len(db_questions)}")

    # If no questions are found, display a message
    if not db_questions:
        question_display = "## ⚠️ No Questions Found in Database."
    else:
        # Build question display using '\n'.join() to avoid Markdown rendering issues
        question_list = ["## 📜 All Quiz Questions"]
        for idx, (category, question, option1, option2, option3, option4, correct) in enumerate(db_questions, start=1):
            question_list.append(f"""
**{idx}. {question}**  
🏷️ **Category:** {category}  
✅ A) {option1}  
✅ B) {option2}  
✅ C) {option3}  
✅ D) {option4}  
🎯 **Correct Answer:** {correct}  
---
""")
        
        question_display = "\n".join(question_list)  # Convert list to string

    # Debugging: Print the Markdown output to check for errors
    print(question_display)

    # Clear and update the current view to force a UI refresh
    current_view.objects.clear()
    current_view.objects = [pn.Column(
        pn.pane.Markdown(question_display, styles={"font-size": "18px", "font-family": "Comic Sans Ms", "text-align": "left"}),
        pn.Spacer(height=20),
        pn.widgets.Button(
            name="🔙 Back to Manage Questions", 
            button_type="light", 
            width=300, 
            on_click=lambda event: show_manage_questions()
        ),
        align="center",
        styles={"background-color": "#FFF8DC", "padding": "20px", "border-radius": "15px", "box-shadow": "5px 5px 15px gray"}
    )]


def get_db_questions():
    """Fetches questions from the database."""
    conn = connect_quiz_db()
    db_questions = []

    if conn:
        try:
            cursor = conn.cursor()
            cursor.execute("SELECT category, question, option1, option2, option3, option4, correct_answer FROM quiz_questions")
            db_questions = cursor.fetchall()
            print(f"✅ Fetched {len(db_questions)} Questions.")  # Debugging
        except Exception as e:
            print(f"❌ Database Error: {e}")
        finally:
            conn.close()
    
    return db_questions



def show_add_question():
    """Displays the Add Question page where the admin can enter a new question and save it."""
    
    # Input fields
    category_input = pn.widgets.TextInput(name="📂 Category", placeholder="Enter category", width=400)
    question_input = pn.widgets.TextAreaInput(name="❓ Question", placeholder="Enter question", width=400, height=100)
    option1_input = pn.widgets.TextInput(name="🅰 Option 1", placeholder="Enter option 1", width=400)
    option2_input = pn.widgets.TextInput(name="🅱 Option 2", placeholder="Enter option 2", width=400)
    option3_input = pn.widgets.TextInput(name="🆎 Option 3", placeholder="Enter option 3", width=400)
    option4_input = pn.widgets.TextInput(name="🅾 Option 4", placeholder="Enter option 4", width=400)
    correct_answer_input = pn.widgets.TextInput(name="✅ Correct Answer", placeholder="Enter correct answer", width=400)

    # Save button
    save_btn = pn.widgets.Button(name="💾 Save Question", button_type="success", width=250)
    
    # Status message
    save_status = pn.pane.Markdown("", styles={"font-size": "16px", "color": "green"})

    # Function to save the question
    def save_question(event):
        category = category_input.value.strip()
        question = question_input.value.strip()
        option1 = option1_input.value.strip()
        option2 = option2_input.value.strip()
        option3 = option3_input.value.strip()
        option4 = option4_input.value.strip()
        correct_answer = correct_answer_input.value.strip()

        if not all([category, question, option1, option2, option3, option4, correct_answer]):
            save_status.object = "❌ All fields are required!"
            save_status.styles = {"color": "red"}
            return
        
        conn = connect_quiz_db()
        if conn:
            try:
                cursor = conn.cursor()
                cursor.execute("""
                    INSERT INTO quiz_questions (category, question, option1, option2, option3, option4, correct_answer) 
                    VALUES (%s, %s, %s, %s, %s, %s, %s)
                """, (category, question, option1, option2, option3, option4, correct_answer))
                conn.commit()
                save_status.object = "✅ Question added successfully!"
                save_status.styles = {"color": "green"}
            except mysql.connector.Error as err:
                save_status.object = f"❌ Database Error: {err}"
                save_status.styles = {"color": "red"}
            finally:
                conn.close()

    save_btn.on_click(save_question)

    # Back to Manage Questions
    back_btn = pn.widgets.Button(name="🔙 Back to Manage Questions", button_type="light", width=250)
    back_btn.on_click(lambda event: show_manage_questions())


    # Layout
    add_question_page = pn.Column(
        pn.pane.Markdown("## ➕ Add a New Question", styles={"font-size": "30px", "font-family": "Garamond", "font-weight": "bold", "text-align": "center"}),
        pn.Spacer(height=20),
        category_input,
        question_input,
        option1_input,
        option2_input,
        option3_input,
        option4_input,
        correct_answer_input,
        pn.Spacer(height=20),
        save_btn,
        pn.Spacer(height=10),
        save_status,
        pn.Spacer(height=20),
        back_btn,
        align="center",
        styles={"background-color": "#F5F5F5", "padding": "30px", "border-radius": "15px", "box-shadow": "5px 5px 15px gray"}
    )

    current_view.objects = [add_question_page]

def show_edit_question():
    """Displays the Edit Question page where the admin can select a category and modify an existing question."""

    # Dropdown for category selection
    categories = get_all_categories()  # Fetch categories from the database
    category_dropdown = pn.widgets.Select(name="📂 Select Category", options=categories, width=300)

    # Dropdown for selecting a question (this will be updated dynamically)
    question_dropdown = pn.widgets.Select(name="📝 Select Question", options=[], width=500)

    # Input fields for editing
    category_input = pn.widgets.TextInput(name="📂 Category", width=400, disabled=True)  # Disabled since it's selected first
    question_input = pn.widgets.TextInput(name="❓ Question", placeholder="Enter question", width=400)
    option1_input = pn.widgets.TextInput(name="🅰️ Option 1", placeholder="Enter first option", width=400)
    option2_input = pn.widgets.TextInput(name="🅱️ Option 2", placeholder="Enter second option", width=400)
    option3_input = pn.widgets.TextInput(name="🆎 Option 3", placeholder="Enter third option", width=400)
    option4_input = pn.widgets.TextInput(name="🅾️ Option 4", placeholder="Enter fourth option", width=400)
    correct_answer_input = pn.widgets.TextInput(name="✅ Correct Answer", placeholder="Enter correct answer", width=400)

    # Status message
    update_status = pn.pane.Markdown("", styles={"font-size": "16px", "color": "green"})

    # Function to update question dropdown based on selected category
    def update_questions(event):
        selected_category = category_dropdown.value
        questions = get_questions_by_category(selected_category)  # Fetch questions from the database
        question_dropdown.options = questions if questions else ["No questions available"]

    category_dropdown.param.watch(update_questions, "value")

    # Function to prefill fields when a question is selected
    def load_question(event):
        selected_question = question_dropdown.value
        if not selected_question or selected_question == "No questions available":
            return

        conn = connect_quiz_db()
        if conn:
            try:
                cursor = conn.cursor(dictionary=True)
                cursor.execute("SELECT * FROM quiz_questions WHERE question = %s", (selected_question,))
                question_data = cursor.fetchone()

                if question_data:
                    category_input.value = question_data['category']
                    question_input.value = question_data['question']
                    option1_input.value = question_data['option1']
                    option2_input.value = question_data['option2']
                    option3_input.value = question_data['option3']
                    option4_input.value = question_data['option4']
                    correct_answer_input.value = question_data['correct_answer']

            except Exception as e:
                update_status.object = f"❌ Error loading question: {e}"
            finally:
                conn.close()

    # Load question details when selected
    question_dropdown.param.watch(load_question, "value")

    # Function to update question in the database
    def update_question(event):
        selected_question = question_dropdown.value
        if not selected_question or selected_question == "No questions available":
            update_status.object = "❌ Please select a question!"
            return

        conn = connect_quiz_db()
        if conn:
            try:
                cursor = conn.cursor()
                cursor.execute("""
                    UPDATE quiz_questions 
                    SET question=%s, option1=%s, option2=%s, option3=%s, option4=%s, correct_answer=%s
                    WHERE question=%s
                """, (
                    question_input.value, 
                    option1_input.value, option2_input.value, 
                    option3_input.value, option4_input.value, 
                    correct_answer_input.value, selected_question
                ))
                conn.commit()
                update_status.object = "✅ Question updated successfully!"
                update_questions(None)  # Refresh question list
            except Exception as e:
                update_status.object = f"❌ Database Error: {e}"
            finally:
                conn.close()

    # Update button
    update_btn = pn.widgets.Button(name="💾 Save Changes", button_type="success", width=250)
    update_btn.on_click(update_question)

    # Back button
    back_btn = pn.widgets.Button(name="🔙 Back to Manage Questions", button_type="light", width=250)
    back_btn.on_click(lambda event: show_manage_questions())

    # Layout
    edit_question_page = pn.Column(
        pn.pane.Markdown("## ✏️ Edit a Question", styles={"font-size": "30px", "font-family": "Garamond", "font-weight": "bold", "text-align": "center"}),
        pn.Spacer(height=20),
        category_dropdown,
        pn.Spacer(height=10),
        question_dropdown,
        pn.Spacer(height=10),
        category_input,
        question_input,
        option1_input,
        option2_input,
        option3_input,
        option4_input,
        correct_answer_input,
        pn.Spacer(height=10),
        update_btn,
        update_status,
        pn.Spacer(height=10),
        back_btn,
        align="center",
        styles={"background-color": "#F5F5F5", "padding": "20px", "border-radius": "15px", "box-shadow": "5px 5px 15px gray"}
    )

    current_view.objects = [edit_question_page]


def show_delete_question():
    """Displays the Delete Question page where the admin can select a category and delete a question."""

    # Dropdown for category selection
    categories = get_all_categories()  # Fetch categories from the database
    category_dropdown = pn.widgets.Select(name="📂 Select Category", options=categories, width=300)

    # Dropdown for selecting a question (this will be updated dynamically)
    question_dropdown = pn.widgets.Select(name="❓ Select Question", options=[], width=500)

    # Function to update question dropdown based on selected category
    def update_questions(event):
        selected_category = category_dropdown.value
        questions = get_questions_by_category(selected_category)  # Fetch questions from the database
        question_dropdown.options = questions if questions else ["No questions available"]

    category_dropdown.param.watch(update_questions, "value")

    # Delete button
    delete_btn = pn.widgets.Button(name="🗑️ Delete Question", button_type="danger", width=200)

    # Status message
    delete_status = pn.pane.Markdown("", styles={"font-size": "16px", "color": "red"})

    # Function to delete the selected question
    def delete_selected_question(event):
        selected_question = question_dropdown.value
        if selected_question and selected_question != "No questions available":
            success = delete_question_from_db(selected_question)
            if success:
                delete_status.object = "✅ Question deleted successfully!"
                update_questions(None)  # Refresh question list
            else:
                delete_status.object = "❌ Error deleting question."

    delete_btn.on_click(delete_selected_question)

    # Back button
    back_btn = pn.widgets.Button(name="🔙 Back to Manage Questions", button_type="light", width=250)
    back_btn.on_click(lambda event: show_manage_questions())

    # Layout
    delete_question_page = pn.Column(
        pn.pane.Markdown("## 🗑️ Delete a Question", styles={"font-size": "30px", "font-family": "Garamond", "font-weight": "bold", "text-align": "center"}),
        pn.Spacer(height=20),
        category_dropdown,
        pn.Spacer(height=10),
        question_dropdown,
        pn.Spacer(height=10),
        delete_btn,
        pn.Spacer(height=10),
        delete_status,
        pn.Spacer(height=20),
        back_btn,
        align="center",
        styles={"background-color": "#F5F5F5", "padding": "20px", "border-radius": "15px", "box-shadow": "5px 5px 15px gray"}
    )

    current_view.objects = [delete_question_page]

# Functions to interact with the database

def get_all_categories():
    """Fetches all available categories from the database."""
    conn = connect_quiz_db()
    categories = []
    if conn:
        try:
            cursor = conn.cursor()
            cursor.execute("SELECT DISTINCT category FROM quiz_questions")
            categories = [row[0] for row in cursor.fetchall()]
        except Exception as e:
            print(f"Error fetching categories: {e}")
        finally:
            conn.close()
    return categories

def get_questions_by_category(category):
    """Fetches all questions under a given category from the database."""
    conn = connect_quiz_db()
    questions = []
    if conn:
        try:
            cursor = conn.cursor()
            cursor.execute("SELECT question FROM quiz_questions WHERE category = %s", (category,))
            questions = [row[0] for row in cursor.fetchall()]
        except Exception as e:
            print(f"Error fetching questions: {e}")
        finally:
            conn.close()
    return questions

def delete_question_from_db(question):
    """Deletes a specific question from the database."""
    conn = connect_quiz_db()
    success = False
    if conn:
        try:
            cursor = conn.cursor()
            cursor.execute("DELETE FROM quiz_questions WHERE question = %s", (question,))
            conn.commit()
            success = True
        except Exception as e:
            print(f"Error deleting question: {e}")
        finally:
            conn.close()
    return success

#####manage players---------------------


def manage_players():
    """Displays the manage players options centered with a different font style."""
    manage_players_page = pn.Column(
        pn.pane.Markdown("## 🧑‍💻 Manage Players", styles={"font-size": "30px", "font-weight": "bold", "font-family": "Garamond"}),
        pn.Spacer(height=20),
        pn.widgets.Button(name="👁️ View All Players", button_type="primary", width=300, on_click=view_all_players),
        pn.widgets.Button(name="🗑️ Delete a Player", button_type="danger", width=300, on_click=delete_player_prompt),
        pn.widgets.Button(name="🔍 Search a Player", button_type="primary", width=300, on_click=search_player_prompt),
        pn.widgets.Button(name="🔙 Back to Main", button_type="light", width=300, on_click=lambda event: show_admin_dashboard()),  # Ensure this function is correct
        align="center",
        styles={"background-color": "#FFF8DC", "padding": "20px", "border-radius": "15px", "box-shadow": "5px 5px 15px gray"}
    )

    current_view.objects = [manage_players_page]

def get_all_players():
    """Fetches all players from the leaderboard table."""
    conn = connect_quiz_db()
    players = []

    if conn:
        try:
            cursor = conn.cursor()
            cursor.execute("SELECT id, player_name, category, score FROM leaderboard")
            players = cursor.fetchall()
            print(f"✅ Fetched {len(players)} Players.")  # Debugging
        except Exception as e:
            print(f"❌ Database Error: {e}")
        finally:
            conn.close()

    return players


def view_all_players(event):
    """Displays all players in the leaderboard."""
    players = get_all_players()

    player_display = "## 📋 All Players\n"
    if not players:
        player_display += "No players found."
    else:
        for idx, (player_id, player_name, category, score) in enumerate(players, start=1):
            player_display += f"**{idx}. {player_name}** - Category: {category} - Score: {score}\n"

    players_page = pn.Column(
        pn.pane.Markdown(player_display, styles={"font-size": "18px", "font-family": "Garamond", "text-align": "left"}),
        pn.Spacer(height=20),
        pn.widgets.Button(name="🔙 Back to Manage Players", button_type="light", width=300, on_click=lambda event: manage_players()),
        align="center",
        styles={"background-color": "#FFF8DC", "padding": "20px", "border-radius": "15px", "box-shadow": "5px 5px 15px gray"}
    )

    current_view.objects = [players_page]

def show_main_page():
    """Function to go back to the main page."""
    # You can add your code here to show the main page
    print("Back to Main Page")  # Replace with actual main page display code

def delete_player(player_name):
    """Deletes a player from the leaderboard table by player_name."""
    conn = connect_quiz_db()

    if conn:
        try:
            cursor = conn.cursor()
            cursor.execute("DELETE FROM leaderboard WHERE player_name = %s", (player_name,))
            conn.commit()

            if cursor.rowcount > 0:
                print(f"✅ Successfully deleted player: {player_name}")
            else:
                print(f"⚠️ Player '{player_name}' not found in the database.")

        except Exception as e:
            print(f"❌ Database Error: {e}")
        finally:
            conn.close()

def delete_player_prompt(event=None):
    """Prompt to delete a player."""
    
    # Input Field for Player Name
    player_name_input = pn.widgets.TextInput(name="Player Name", placeholder="Enter player name to delete")

    # Delete Button
    delete_button = pn.widgets.Button(name="❌ Delete Player", button_type="danger", width=200)

    # Success / Error Message Pane
    message_pane = pn.pane.Markdown("", styles={"font-size": "18px", "color": "red", "font-weight": "bold"})

    def delete_player_action(event):
        player_name = player_name_input.value.strip()
        if player_name:
            delete_player(player_name)
            message_pane.object = f"✅ Player '{player_name}' deleted successfully!"
            player_name_input.value = ""  # Clear input field
        else:
            message_pane.object = "⚠️ Please enter a valid player name."

    delete_button.on_click(delete_player_action)

    # Delete Player Page UI
    delete_page = pn.Column(
        pn.pane.Markdown("## 🗑️ Delete a Player", styles={"font-size": "22px", "font-weight": "bold"}),
        player_name_input,
        delete_button,
        message_pane,  # Display success/error messages
        pn.Spacer(height=20),
        pn.widgets.Button(name="🔙 Back to Manage Players", button_type="light", width=300, on_click=lambda event: manage_players()),
        align="center",
        styles={"background-color": "#FFF8DC", "padding": "20px", "border-radius": "15px", "box-shadow": "5px 5px 15px gray"}
    )

    current_view.objects = [delete_page]

def search_player(player_name):
    """Searches for a player in the leaderboard table (case-insensitive)."""
    conn = connect_quiz_db()
    
    if conn:
        try:
            cursor = conn.cursor()
            query = "SELECT id, player_name, category, score FROM leaderboard WHERE LOWER(player_name) = LOWER(%s)"
            cursor.execute(query, (player_name,))
            result = cursor.fetchone()  # Fetch only one matching result

            if result:
                print(f"✅ Player found: {result}")
            else:
                print(f"⚠️ No player found with name '{player_name}'.")

            return result

        except Exception as e:
            print(f"❌ Database Error: {e}")
            return None
        finally:
            conn.close()



def search_player_prompt(event=None):
    """Prompt to search for a player."""

    # Input Field for Player Name
    player_name_input = pn.widgets.TextInput(name="Player Name", placeholder="Enter player name to search")

    # Search Button
    search_button = pn.widgets.Button(name="🔍 Search Player", button_type="primary", width=200)

    # Result Display Pane
    message_pane = pn.pane.Markdown("", styles={"font-size": "18px", "color": "black", "font-weight": "bold"})

    def search_player_action(event):
        player_name = player_name_input.value.strip()
        if player_name:
            player = search_player(player_name)
            if player:
                player_display = f"""
                ## 🏆 Player Found  
                **👤 Name:** {player[1]}  
                **🏷️ Category:** {player[2]}  
                **🎯 Score:** {player[3]}  
                """
            else:
                player_display = f"❌ No player found with name **{player_name}**."

            message_pane.object = player_display  # Update UI dynamically

        else:
            message_pane.object = "⚠️ Please enter a valid player name."

    search_button.on_click(search_player_action)

    # Search Player Page UI
    search_page = pn.Column(
        pn.pane.Markdown("## 🔍 Search a Player", styles={"font-size": "22px", "font-weight": "bold"}),
        player_name_input,
        search_button,
        message_pane,  # Display search results here
        pn.Spacer(height=20),
        pn.widgets.Button(name="🔙 Back to Manage Players", button_type="light", width=300, on_click=lambda event: manage_players()),
        align="center",
        styles={"background-color": "#FFF8DC", "padding": "20px", "border-radius": "15px", "box-shadow": "5px 5px 15px gray"}
    )

    current_view.objects = [search_page]



# ========== View Management ==========
def show_welcome():
    current_view.objects = [welcome_page]

def show_player_login(event=None):
    current_view.objects = [player_login_page]

def show_admin_login(event=None):
    current_view.objects = [admin_login_page]

def return_home(event=None):
    show_welcome()

#

#-----------welcome page-------------
# Adjusted Spacer for More Right Shift
import panel as pn

# Define actions for the buttons
def go_to_player_login(event=None):
    print("Navigating to Player Login...")
    current_view.objects = [player_login_page]  # Assuming `player_login_page` exists

def go_to_admin_login(event=None):
    print("Navigating to Admin Login...")
    current_view.objects = [admin_login_page]  # Assuming `admin_login_page` exists

player_login_btn = pn.widgets.Button(
    name="Player Login", 
    button_type="primary",  
    styles={"background": "#9370DB", "color": "black", "font-size": "24px", "font-weight": "bold"}
)

admin_login_btn = pn.widgets.Button(
    name="Admin Login", 
    button_type="primary",  
    styles={"background": "#9370DB", "color": "black", "font-size": "24px", "font-weight": "bold"}
)

# Bind button clicks to functions
player_login_btn.on_click(go_to_player_login)
admin_login_btn.on_click(go_to_admin_login)

# Resolve the image path
image_path = Path("C:/Users/HP/Documents/Python lab programs/images/Welcome.png").resolve()

# Enable Panel extensions
pn.extension()

# Create an image pane
image_pane = pn.pane.Image(image_path, width=175)  # Adjust width as needed

# Layout for the welcome page
welcome_page = pn.Row(
    pn.Spacer(width=200),  # 🔥 Move content slightly to the right
    pn.Column(
        pn.Spacer(height=50),  # Adjust vertical spacing
        pn.pane.HTML("""
            <div style="text-align: center;">
                <h1 style="
                    font-size: 50px; 
                    font-weight: bold; 
                    font-family: 'Poppins', Garamond;
                    background: linear-gradient(to right, #ff8c00, #ff0080);
                    -webkit-background-clip: text;
                    -webkit-text-fill-color: transparent;
                ">🚀 WELCOME TO QUIZIVERSE 🎉</h1>
                <p style="
                    font-size: 22px; 
                    font-family: 'Poppins', Garamond;
                    font-weight: bold; 
                    color: #5D3FD3;
                "> 🧠 Unlock your genius, one question at a time! ✨ </p>
            </div>
        """),
        pn.Spacer(height=20),
        pn.Row(player_login_btn, admin_login_btn, align="center"),  # ✅ Buttons side by side
        pn.Spacer(height=50),  # 🔥 Space between buttons and image
        pn.Row(image_pane, align="center"),  # ✅ Image now centered below buttons
        pn.Spacer(height=100),
        align="center"
    ),
    sizing_mode="stretch_width"  # Ensures full width without scrolling
)

# Create the player login box
pn.extension(raw_css=[
    """
    body {
        background-color: #FAD2E1 !important; /* Soft Beige-Brown */
    }
    """
])

player_login_page = pn.Row(
    pn.Spacer(width=400),  # Adds horizontal spacing if needed
    pn.Column(
        pn.layout.VSpacer(),  # Pushes content downward
        pn.pane.HTML(
            "<h2 style='text-align:center; font-size:30px; font-family:Garamond;'>"
            "🎮 Player Login 🎮</h2>"
        ),
        name_input,
        category_dropdown,
        start_button,
        status_pane,
        return_home_btn,
        pn.layout.VSpacer(),  # Pushes content upward
        align="center",
        width=400,
        styles={
            "background-color": "#C4A484",  # Light Brown for Box
            "border": "3px solid lavender",
            "border-radius": "15px",
            "padding": "20px",
            "box-shadow": "5px 5px 10px gray",
            "font-family": "Comic Sans MS",
            "margin": "auto",  # Centers horizontally
            "display": "flex",
            "flex-direction": "column",
            "align-items": "center"
        }
    ),
    pn.Spacer(width=50)  # Adds horizontal spacing if needed
)

# Updating button styles
start_button.styles = {"font-size": "18px", "font-weight": "bold"}  # Bigger & bolder font
return_home_btn.styles = {"font-size": "18px", "font-weight": "bold"}  # Bigger & bolder font



# Admin Login Page (corrected version)
admin_login_page = pn.Column(
    pn.Spacer(height=100),
    pn.pane.HTML(
        "<h2 style='text-align:center; font-size:40px; font-weight:bold;'>Admin Login</h2>", 
        width=600,
        align='center'
    ),
    pn.Spacer(height=30),
    pn.Row(pn.Spacer(width=200), admin_user, pn.Spacer(width=350)),
    pn.Row(pn.Spacer(width=200), admin_pass, pn.Spacer(width=350)),
    pn.Row(pn.Spacer(width=200), admin_confirm_pass, pn.Spacer(width=350)),
    pn.Row(pn.Spacer(width=200), admin_login_button, pn.Spacer(width=350)),
    pn.Row(pn.Spacer(width=200), admin_status, pn.Spacer(width=350)),
    pn.Spacer(height=150),
    return_home_btn,
    sizing_mode='stretch_width',
    align='center'
)

# Quiz App Page
quiz_app = pn.Column(
    pn.Spacer(height=50),
    question_pane,
    *[pn.Row(btn, align="center") for btn in option_buttons],
    pn.Spacer(height=20),
    return_home_btn,
    align="center",
    width=600
)


# Admin Dashboard (corrected version)
# Create the admin login form inside a properly wrapped box
admin_login_page = pn.Column(
    pn.pane.HTML(
        "<h2 style='font-size:40px; font-weight:bold; font-family:Garamond; text-align:center;'>"
        "🔑 Admin Login 🔑</h2>",  # Added emojis
        width=400,
        align='center'
    ),
    pn.Spacer(height=30),
    admin_user,
    pn.Spacer(height=10),
    admin_pass,
    pn.Spacer(height=10),
    admin_confirm_pass, 
    pn.Spacer(height=20),
    admin_login_button,
    pn.Spacer(height=20),
    admin_status,
    pn.Spacer(height=30),
    return_home_btn,
    width=400,
    align='center',
    styles={  
        "background-color": "#AEC6CF",  # Pastel Blue inside the box  
        "border": "3px solid lavender",  # Lavender Border  
        "border-radius": "15px",  
        "padding": "30px",  
        "box-shadow": "5px 5px 15px gray",  
        "font-family": "Comic Sans Ms",  
        "display": "flex",
        "flex-direction": "column",
        "align-items": "center"
    }
)

# Center the whole box in the middle of the screen
admin_login_wrapper = pn.Column(
    pn.Spacer(height=50),  # Keeps some spacing above
    admin_login_page,
    align="center",
    styles={
        "position": "absolute",  
        "top": "50%",  # Centers vertically  
        "left": "50%",  # Centers horizontally  
        "transform": "translate(-50%, -50%)",  # Ensures perfect centering  
    }
)

# Update button styles for bold and bigger font
admin_login_button.styles = {"font-size": "18px", "font-weight": "bold", "font-family": "Comic Sans Ms"}
return_home_btn.styles = {"font-size": "18px", "font-weight": "bold", "font-family": "Comic Sans Ms"}

# Admin Dashboard

# ========== Event Bindings ==========
start_button.on_click(start_quiz)
for i, btn in enumerate(option_buttons):
    btn.on_click(lambda event, i=i: check_answer(i))
    
admin_login_button.on_click(admin_login)
player_login_btn.on_click(show_player_login)
admin_login_btn.on_click(show_admin_login)
return_home_btn.on_click(return_home)

# ========== App Initialization ==========
current_view = pn.Column(welcome_page)
show_welcome()

# ========== Serve App ==========
pn.serve(current_view, show=True)

Launching server at http://localhost:53813
